In [ ]:
from sunpy import timeseries as ts
from sunpy.net import Fido
from sunpy.net import attrs as a
from netCDF4 import Dataset
import datetime as dt
import numpy as np

In [ ]:
def create_data(file_goes15, tstart, tend):
    if len(file_goes15.data) <1:
        print('no_data')
        return -1
    rootgrp = Dataset(file_goes15.data[0], 'r')
    if len(file_goes15.data) > 1:
        rootgrp = Dataset(file_goes15.data[1], 'r')
        rootgrp1 = Dataset(file_goes15.data[0], 'r')
        #check order
        if rootgrp1.variables['time'][0] < rootgrp.variables['time'][0]:
            rootgrp, rootgrp1 = rootgrp1, rootgrp
        quality_b1 = rootgrp1.variables['b_flags'][:]
        quality_a1 = rootgrp1.variables['a_flags'][:]
    quality_b = rootgrp.variables['b_flags'][:]
    quality_a = rootgrp.variables['a_flags'][:]

    #upload whole file
    times = []

    goes_15 = ts.TimeSeries(file_goes15, concatenate=True)
    goes_fl = goes_15.truncate(tstart, tend)
    #k= goes_fl.data.flags
    #print(k)
    table = goes_fl.to_table()
    #find the index of truncated data
    if len(goes_fl.index) ==0:
        print('error')
        return -1
    start_date = goes_fl.index[0]
    end_date = goes_fl.index[-1]
    data  = dt.datetime(start_date.year, start_date.month, start_date.day, start_date.hour, start_date.minute, start_date.second, microsecond = start_date.microsecond, tzinfo=dt.timezone.utc)
    data_end = dt.datetime(end_date.year, end_date.month, end_date.day, end_date.hour, end_date.minute, end_date.second, microsecond= end_date.microsecond, tzinfo=dt.timezone.utc)

    dt_start = (data - dt.datetime(1970, 1, 1, tzinfo=dt.timezone.utc)).total_seconds()
    dt_end= (data_end - dt.datetime(1970, 1, 1, tzinfo = dt.timezone.utc)).total_seconds()

    times_index = np.where(rootgrp.variables['time'][:].data==dt_start)
    if len(times_index[0])==0:

        print('error')
        return -1
    if len(file_goes15.data) == 1:
        times_index_end = np.where(rootgrp.variables['time'][:].data == dt_end)
    else:
        times_index_end = np.where(rootgrp1.variables['time'][:].data == dt_end)

    s = times_index_end[0]
    k= len(s)

    if len(times_index_end[0])==0:
        print('error')
        return -1

    if len(file_goes15.data) ==1:
        table['b_flags'] = quality_b[int(times_index[0]):int(times_index_end[0])+1]
        table['a_flags'] = quality_a[int(times_index[0]):int(times_index_end[0])+1]
    else:
        ind0 = int(times_index[0])
        ind1 =int(times_index_end[0])+1
        arr1 = quality_b[ind0:]
        arr2 = quality_b1[:ind1]
        arr = np.concatenate((arr1, arr2))
        table['b_flags'] = arr
        table['a_flags'] = np.concatenate((quality_a[int(times_index[0]):] , quality_a1[:int(times_index_end[0]) + 1]))

    return table



In [ ]:
#start and end dates
tstart = "2012-05-04 00:00"
tend = "2012-05-04 12:00"

#make a request
result = Fido.search(a.Time(tstart, tend), a.Instrument("XRS"))
result_goes15 = Fido.search(a.Time(tstart, tend), a.Instrument("XRS"), a.goes.SatelliteNumber(15))
file_goes15 = Fido.fetch(result_goes15)

# create astropy table with data
table = create_data(file_goes15, tstart, tend)



In [ ]:
# chose indexes with good quality data

indx_good = [ind for ind, value in enumerate ( table["b_flags"]) if value  == 0]